In [1]:
import pandas as pd
import numpy as np
import random
from random import randint
from scipy.linalg import eigh
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import normalized_mutual_info_score as nmi_score

In [2]:
train = pd.read_csv("./compound.csv",names = ["x1", "x2", "label"])
train

,x1,x2,label
0,26.75,22.15,1
1,29.80,22.15,1
2,31.55,21.10,1
3,27.70,20.85,1
4,29.90,19.95,1
5,26.80,19.05,1
6,28.35,18.25,1
7,30.40,17.85,1
8,27.25,16.70,1
9,29.05,16.00,1


In [3]:
train_data = train.drop("label",axis=1)
train_data = train_data.values
train_labels = train["label"].values
for i in range(len(train_data)):
    train_labels[i] = train_labels[i]-1
train_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4,

In [63]:
tr_data_labels = np.c_[train_data,train_labels]
np.random.shuffle(tr_data_labels)
tr_data_labels

train_labels = tr_data_labels[:,2]
train_data = np.delete(tr_data_labels, 2, 1)
print(train_data.shape)
print(train_labels.shape)


(399, 2)
(399,)


In [64]:
N = len(train_data)
dists = [[0 for j in range(N)] for i in range(N)]
diag  = [[0 for j in range(N)] for i in range(N)]
norms = [[0 for j in range(N)] for i in range(N)]

# can improve implementation by using the fact that dists will be symmetric.
for i in range(N):
    for j in range(N):
#         norms[i][j] = np.power(np.linalg.norm(train_data[i]-train_data[j]),2)
        dists[i][j] = np.exp(-np.power(np.linalg.norm(train_data[i]-train_data[j]),2))
        diag[i][i] += dists[i][j]
diag = np.matrix(diag)
dists = np.matrix(dists)


In [65]:
tmp = diag-dists

s = [[0 for j in range(N)] for i in range(N)]
for i in range(N):
#     s[i][i] = np.power(diag[i,i],-0.5)
    s[i][i] = np.sqrt(1/diag[i,i])

d1 = np.matrix(s)

# for i in range(N):
#     s[i][i] = np.power(diag[i,i],0.5)
d2 = np.matrix(s)

L = np.matmul(np.matmul(d1,tmp),d2)
L

matrix([[  5.10560005e-001,  -5.88883694e-275,  -7.97125543e-232, ...,
          -1.05159561e-036,  -1.62508391e-210,  -2.84462113e-008],
        [ -5.88883694e-275,   3.03153910e-001,  -1.01317822e-004, ...,
          -2.52256306e-310,  -2.94775284e-020,  -6.93504831e-296],
        [ -7.97125543e-232,  -1.01317822e-004,   9.10032406e-001, ...,
          -3.08597500e-253,  -1.46077118e-008,  -8.43421773e-258],
        ..., 
        [ -1.05159561e-036,  -2.52256306e-310,  -3.08597500e-253, ...,
           8.48325122e-001,  -1.22654650e-203,  -1.85961564e-075],
        [ -1.62508391e-210,  -2.94775284e-020,  -1.46077118e-008, ...,
          -1.22654650e-203,   8.52676842e-001,  -3.64383076e-248],
        [ -2.84462113e-008,  -6.93504831e-296,  -8.43421773e-258, ...,
          -1.85961564e-075,  -3.64383076e-248,   6.21299000e-001]])

In [66]:
eigen_vals, eigen_vecs = eigh(L)
eigen_vals, eigen_vecs

(array([ -4.44170217e-16,   2.19864480e-13,   1.18642348e-05,
          1.38469016e-03,   1.97906421e-03,   2.15717342e-03,
          2.38205475e-03,   3.65071329e-03,   4.90658900e-03,
          4.96999456e-03,   7.13862202e-03,   7.50962523e-03,
          7.61680206e-03,   9.98035595e-03,   1.17945805e-02,
          1.30299815e-02,   1.46355976e-02,   1.72737728e-02,
          1.81768979e-02,   1.83224217e-02,   1.99142124e-02,
          2.13761214e-02,   2.27836386e-02,   2.82378260e-02,
          2.94279804e-02,   3.17343434e-02,   3.19124377e-02,
          3.27660252e-02,   3.48672623e-02,   3.57750921e-02,
          3.64341408e-02,   3.96419751e-02,   4.14765602e-02,
          4.36532794e-02,   4.43458839e-02,   5.98580912e-02,
          6.13863708e-02,   6.37850765e-02,   6.43165572e-02,
          7.15526472e-02,   7.15969968e-02,   7.18854006e-02,
          7.49698063e-02,   7.78982068e-02,   7.91459726e-02,
          8.19090903e-02,   8.31622206e-02,   8.63281791e-02,
        

In [67]:
k = 3
# V = eigen_vecs[:,N-k:]
V = eigen_vecs[:,:k]
print(V)

[[ -2.96955808e-02  -2.22190126e-02  -7.58529724e-02]
 [ -2.47412954e-02   3.30665946e-02   7.46904039e-10]
 [ -6.88569554e-02   9.20269126e-02   2.07866241e-09]
 ..., 
 [ -5.33438951e-02  -3.99133055e-02   3.26590559e-02]
 [ -5.38090542e-02   7.19154819e-02   1.62398453e-09]
 [ -3.37592917e-02  -2.52595876e-02  -8.62716491e-02]]


In [151]:
def k_means(train_data, train_labels, eps_dis = 1, d = 2, k = 6):
    # initialize means to 10 random points
    N = len(train_data)
    means_indices = np.random.randint(low=0,high=len(train_data),size=k)
    #means_indices = [36,31,28,24,21,19,18,26,34,30]
    means_new = np.zeros((k,d))
    means_old = np.array([train_data[i] for i in means_indices])
    eps_dist = eps_dis
    diff = means_new-means_old
    lst = []
    for i in range(k):
        lst.append(np.linalg.norm(diff[i]))
    dist = np.max(np.array(lst))
#     dist = np.linalg.norm(means_new-means_old)
    #dist = 100
    #dist2 = 0

    classes = []
    mean_dists = [[0 for i in range(len(train_data))] for j in range(k)]
    pbar = tqdm(total = 200+1)

    cntr = 0
    while dist >= eps_dist:
        cntr += 1
        b = np.zeros((k,len(train_data)))
        mean_dists = [[0 for i in range(len(train_data))] for j in range(k)]

        for i in range(k):
            for j in range(len(train_data)):
                mean_dists[i][j] = np.linalg.norm(train_data[j]-means_old[i])
        mean_dists = np.array(mean_dists)

        for j in range(len(train_data)):
            b[np.argmin(mean_dists[:,j])][j] = 1

        b_sum = np.sum(b,axis=1)

        for i in range(k):
            for j in range(len(train_data)):
                means_new[i] = np.add(means_new[i],b[i][j]*train_data[j])
            means_new[i] = np.divide(means_new[i],b_sum[i])

        #dist = np.linalg.norm(means_new-means_old)
        diff = means_new-means_old
        lst = []
        for i in range(k):
            lst.append(np.linalg.norm(diff[i]))
        dist = np.max(np.array(lst))
        means_old = means_new
        means_new =  np.zeros((k,d))
        pbar.update(1)
        print("Iter %d: dist = " %(cntr),dist)#eps_dist,np.count_nonzero(means_old))

    #print(dist,eps_dist)
    # finding diameters of clusters
    diams = [0 for i in range(k)]
    b = np.zeros((k,len(train_data)))
    for i in range(k):
            for j in range(len(train_data)):
                mean_dists[i][j] = np.linalg.norm(train_data[j]-means_old[i])
    mean_dists = np.array(mean_dists)

    for j in range(len(train_data)):
            b[np.argmin(mean_dists[:,j])][j] = 1
    for i in range(k):
        for j in range(N):
            for l in range(N):
                if b[i][j] == 1 and b[i][l] == 1:
                    t = np.linalg.norm(train_data[j]-train_data[l])
                    if diams[i] < t:
                        diams[i] = t
                
    
    # predicting labels
    final_labels = [0 for i in range(N)]
    for j in range(N):
        tmp = np.inf
        lbl = -1
        for i in range(k):
            dis = np.linalg.norm(train_data[j]-means_old[i])
            if dis < tmp:
                tmp = dis
                lbl = i
        final_labels[j] = lbl
    print("means: ",means_old)
    val = [0 for i in range(k)]
    for i in range(N):
                val[int(final_labels[i])] += np.power(np.linalg.norm(train_data[i]-means_old[final_labels[i]]),2)

    print("\nsq_dists: ",val)
    print("\ntotal sq_dists: ", sum(val))
    print("\ndiameters: ",diams)
    return nmi_score(train_labels, final_labels)

In [152]:
K = 3
print("K = 3")
V = eigen_vecs[:,:K]
nmi = k_means(train_data = V, train_labels=train_labels, eps_dis = 0.001, k = 6, d = K)
print("\nK = nmi: ",nmi)

K = 3


Iter 1: dist =  0.0272224074862
Iter 2: dist =  0.000301885343177
means:  [[ -5.26711208e-02   7.03946406e-02   1.59018207e-09]
 [ -6.45457019e-02   8.62649479e-02   1.94856389e-09]
 [ -2.15925924e-02   2.88583717e-02   6.19862141e-10]
 [ -3.67546347e-02  -2.75007819e-02  -9.36901136e-02]
 [ -5.76130953e-02  -4.31076333e-02   3.52510200e-02]
 [ -4.87376382e-02  -3.64667828e-02   2.98270630e-02]]

sq_dists:  [0.001252806124122302, 0.0014712751864703886, 0.00014183641842057835, 0.09654280259346365, 0.0020426243993674109, 0.0013182965800951948]

total sq_dists:  0.102769641302

diameters:  [0.023029955492809766, 0.017510384819188533, 0.0073688935524626673, 0.10804512438134342, 0.01730340104533757, 0.015367617475022414]


K = nmi:  0.783249358315


In [156]:
K = 5
print("K = 5")
# V = eigen_vecs[:,N-K:]
V = eigen_vecs[:,:K]
nmi = k_means(train_data = V, train_labels=train_labels, eps_dis = 0.0001, k = 6, d = K)
print("\nnmi: ",nmi)

K = 5


Iter 1: dist =  0.0829104536
Iter 2: dist =  0.132244405797
Iter 3: dist =  0.0244969669973
Iter 4: dist =  0.0168769782915
Iter 5: dist =  0.00162136186308
Iter 6: dist =  0.00117934389473
Iter 7: dist =  0.000307755808341
Iter 8: dist =  0.0
means:  [[ -3.67546347e-02  -2.75007819e-02  -9.36901136e-02  -2.99889143e-12
   -8.94821580e-12]
 [ -5.42465426e-02  -4.05886900e-02   3.31936570e-02   1.78137752e-14
   -3.68683746e-13]
 [ -2.18308881e-02   2.91768524e-02   6.13260422e-10  -5.79021114e-03
    1.63182102e-02]
 [ -2.10365691e-02   2.81152499e-02   6.35266152e-10  -1.99641404e-01
    1.36328464e-01]
 [ -6.45457019e-02   8.62649479e-02   1.94856389e-09   1.30282426e-02
   -1.06154772e-02]
 [ -5.26711208e-02   7.03946406e-02   1.59018207e-09   1.02446775e-02
   -8.73325307e-03]]

sq_dists:  [0.09654280259346365, 0.0096004576033993597, 0.087231810148753197, 1.0031815515218905, 0.0015377406048018329, 0.0013299389613015051]

total sq_dists:  1.19942430143

diameters:  [0.10804512438134

In [164]:
K = 10
print("K = 10")
# V = eigen_vecs[:,N-K:]
V = eigen_vecs[:,:K]
nmi = k_means(train_data = V, train_labels=train_labels, eps_dis = 0.0002, k = 6, d = K)
print("\nnmi: ",nmi)

K = 10


Iter 1: dist =  0.0520229093996
Iter 2: dist =  0.117808994843
Iter 3: dist =  0.0216136728177
Iter 4: dist =  0.0105207838442
Iter 5: dist =  0.00474376566362
Iter 6: dist =  0.0
means:  [[ -2.11895375e-02   2.83196912e-02   5.91938911e-10  -1.06322775e-01
    8.56978077e-02   9.28836145e-12  -7.08254993e-02   1.72063993e-02
   -2.00228888e-02   7.09021167e-02]
 [ -2.07051860e-02   2.76723583e-02   6.26943811e-10   5.93020113e-03
   -1.06826872e-02  -1.73788577e-11  -1.09337444e-01  -2.06130775e-03
    9.77363676e-01   1.68646744e-01]
 [ -3.58323869e-02  -2.68107318e-02  -9.15330586e-02  -8.18623968e-12
   -9.14225444e-11  -1.16076318e-01   1.56326940e-11  -5.25547692e-12
   -1.09406209e-11   6.10391216e-11]
 [ -5.32769829e-02   7.12043717e-02   1.60646829e-09   8.76183363e-03
   -6.86817391e-03  -8.80874416e-12   6.45576671e-03  -1.22436638e-03
   -1.10320040e-03  -4.67442404e-03]
 [ -3.74964428e-02  -2.80558221e-02  -9.54251362e-02   1.17354086e-12
    5.73898311e-11   8.93231176e-0

#### We see that NMI improves with K (there may be fluctuations due to random initializations).
#### Note: Sometimes nan is encountered. Just rerun that cell in that case. 